# 卷积神经网络
## 知识点
- CNN：为处理图像而设计的神经网络，图像识别、目标检测、语义分割
- 用于计算机视觉的网络特征
    1. 平移不变性
    2. 局部性
- 卷积核/滤波器
- 正确叫法：互相关运算
- 卷积层输出大小：
$$(n_h-k_h+1) \times (n_w-k_w+1).$$
- 特征映射：卷积层的别名
- 感受野：可以通过构建更深层次的网络扩大感受野
- 填充（padding）
- 步幅（stride）

---

In [6]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  #@save
    '''二维互相关运算'''
    h, w = K.shape
    # 卷积核大小
    Y = torch.zeros((X.shape[0] - h + 1, 
                     X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 将X切片，与卷积核对应部分做乘，再利用.sum()求和
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [8]:
X = torch.tensor([[0.0, 1.0, 2.0],
                 [3.0, 4.0, 5.0],
                 [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0],
                 [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [10]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias


In [13]:
'''目标的边缘检测'''
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [14]:
# 手动设计：根据要提取的特征设计卷积核
K = torch.tensor([[1.0, -1.0]])


In [15]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [16]:
corr2d(X.t(), K) # 将输入转置，检测到的特征消失

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [18]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
# 批量大小、通道数、高度、宽度
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核，根据lr手动更新参数
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 11.376
epoch 4, loss 3.720
epoch 6, loss 1.366
epoch 8, loss 0.533
epoch 10, loss 0.214


In [19]:
conv2d.weight.data.reshape((1, 2))
# 很接近此前手动设计的滤波器(1.0, -1.0)

tensor([[ 1.0372, -0.9423]])